In [20]:
import os
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_groq import ChatGroq 

groq_api_key = "gsk_***************************"

db_user = "root"
db_password = "17122004Kim!"  
db_host = "localhost"
db_name = "f1_project"

uri = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}"

db = SQLDatabase.from_uri(uri)

print(f"База подключена Видим таблиц: {len(db.get_usable_table_names())}")

llm = ChatGroq(
    groq_api_key=groq_api_key, 
    model_name="llama-3.3-70b-versatile",
    temperature=0
)

agent_executor = create_sql_agent(
    llm=llm,
    db=db,
    agent_type="openai-tools",
    verbose=True
)
query = "Какая команда набрала больше всего очков в 2020 году?"
print(f"\nВопрос: {query}\n")
try:
    response = agent_executor.invoke(query)
    print(f"\nОтвет AI: {response['output']}")
except Exception as e:
    print(f"Ошибка: {e}")

 База подключена! Видим таблиц: 7
Список таблиц: ['circuits', 'constructor_standings', 'constructors', 'driver_standings', 'drivers', 'races', 'results']

Вопрос: Кто выиграл больше всего гонок в сезоне 2020 года?

> Entering new SQL Agent Executor chain...
Invoking: `sql_db_list_tables` with `{'tool_input': ''}`
responded: circuits, constructors, drivers, races, results, ...
Invoking: `sql_db_schema` with `{'table_names': 'drivers, races, results'}`
responded: CREATE TABLE results (resultId INTEGER, raceId INTEGER, driverId INTEGER, positionOrder INTEGER...)
Invoking: `sql_db_query` with `{'query': 'SELECT d.surname, COUNT(*) as wins FROM results r JOIN drivers d ON r.driverId = d.driverId JOIN races ra ON r.raceId = ra.raceId WHERE ra.year = 2020 AND r.positionOrder = 1 GROUP BY d.surname ORDER BY wins DESC LIMIT 1'}`

[SQL Query Result]: [('Hamilton', 11)]

Final Answer: В сезоне 2020 года больше всего гонок выиграл Льюис Хэмилтон (Lewis Hamilton) — 11 побед.

> Finished chain.
